In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read in the data
clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])

In [6]:
print(clients.head())

   client_id     joined  income  credit_score
0      46109 2002-04-16  172677           527
1      49545 2007-11-14  104564           770
2      41480 2013-03-11  122607           585
3      46180 2001-11-06   43851           562
4      25707 2006-10-06  211422           621

In [8]:
print(loans.sample(10))

     client_id loan_type  loan_amount  repaid  loan_id loan_start   loan_end  \
27       49545    credit         4458       1    10192 2013-02-11 2014-09-11   
62       46180      cash        14007       1    10416 2003-12-05 2005-05-14   
127      32726     other         5325       1    10858 2014-04-02 2016-10-30   
13       46109    credit         3524       0    11867 2005-09-18 2007-08-27   
231      32961      home         4964       0    11631 2004-08-22 2006-06-02   
99       25707    credit         6371       0    11942 2011-01-15 2013-02-21   
327      44387     other         6562       1    10862 2002-12-22 2005-01-18   
178      48177    credit        12216       0    10697 2007-05-07 2009-07-25   
59       41480    credit          811       0    11430 2009-08-02 2012-01-04   
434      26945    credit         3510       0    10196 2002-07-14 2004-08-10   

     rate  
27   3.60  
62   3.95  
127  2.83  
13   5.98  
231  4.17  
99   6.02  
327  0.89  
178  5.85  
59   0.43  

In [9]:
print(payments.sample(10))

      loan_id  payment_amount payment_date  missed
3154    11733             758   2010-09-27       0
1169    11731              77   2008-01-15       0
528     11672             383   2013-06-09       1
466     11026            1590   2014-06-04       1
1059    11287            2370   2007-03-20       1
646     10562            1167   2001-07-16       1
2017    10694            1233   2006-12-29       0
2182    11602            1611   2007-05-25       1
2790    10427            1483   2007-11-04       1
3337    11482            1834   2014-05-01       0

In [38]:
#Manual Feature Engineering Examples
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

print(clients.head())

   client_id     joined  income  credit_score  join_month  log_income
0      46109 2002-04-16  172677           527           4   12.059178
1      49545 2007-11-14  104564           770          11   11.557555
2      41480 2013-03-11  122607           585           3   11.716739
3      46180 2001-11-06   43851           562          11   10.688553
4      25707 2006-10-06  211422           621          10   12.261611

In [44]:
 #Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
print(stats.sort_values('client_id').head())

           mean_loan_amount  max_loan_amount  min_loan_amount
client_id                                                    
25707           7963.950000            13913             1212
26326           7270.062500            13464             1164
26695           7824.722222            14865             2389
26945           7125.933333            14593              653
29841           9813.000000            14837             2778

In [49]:
# Merge with the clients dataframe
dff=clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left')
print(dff.sort_values('client_id').head(10))

    client_id     joined  income  credit_score  join_month  log_income  \
4       25707 2006-10-06  211422           621          10   12.261611   
10      26326 2004-05-06  227920           633           5   12.336750   
19      26695 2004-08-27  174532           680           8   12.069863   
24      26945 2000-11-26  214516           806          11   12.276140   
13      29841 2002-08-17   38354           523           8   10.554614   
6       32726 2006-05-01  235705           730           5   12.370336   
15      32885 2002-05-13   58955           642           5   10.984530   
12      32961 2009-04-07  230341           714           4   12.347316   
7       35089 2010-03-01  131176           771           3   11.784295   
8       35214 2003-08-08   95849           696           8   11.470529   

    mean_loan_amount  max_loan_amount  min_loan_amount  
4        7963.950000            13913             1212  
10       7270.062500            13464             1164  
19       7824.

In [10]:
es = ft.EntitySet(id = 'clients')

In [12]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [13]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [14]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [15]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [16]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [17]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [18]:
# Relationship between clients and previous loans
#ft.Relationship(parent_variable, child_variable)
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [19]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 4]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [21]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
print(primitives[primitives['type'] == 'aggregation'].head(10))

               name         type  \
0     n_most_common  aggregation   
1          num_true  aggregation   
2  time_since_first  aggregation   
3        num_unique  aggregation   
4               sum  aggregation   
5               all  aggregation   
6             trend  aggregation   
7               std  aggregation   
8               max  aggregation   
9               min  aggregation   

                                                           description  
0                             Determines the `n` most common elements.  
1                                  Counts the number of `True` values.  
2   Calculates the time elapsed since the first datetime (in seconds).  
3     Determines the number of distinct values, ignoring `NaN` values.  
4                       Calculates the total addition, ignoring `NaN`.  
5                       Calculates if all values are 'True' in a list.  
6                        Calculates the trend of a variable over time.  
7  Computes the dis

In [22]:
print(primitives[primitives['type'] == 'transform'].head(10))

                       name       type  \
20    greater_than_equal_to  transform   
21                   negate  transform   
22                     hour  transform   
23      time_since_previous  transform   
24        divide_by_feature  transform   
25                     isin  transform   
26                  cum_sum  transform   
27  subtract_numeric_scalar  transform   
28      greater_than_scalar  transform   
29               time_since  transform   

                                                                    description  
20  Determines if values in one list are greater than or equal to another list.  
21                                                     Negates a numeric value.  
22                                     Determines the hour value of a datetime.  
23         Compute the time in seconds since the previous instance of an entry.  
24                                   Divide a scalar by each value in the list.  
25                    Determines whether a va

In [25]:
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = [ 'month'])

In [52]:
#print(features[0:10], feature_names[0:10])

In [31]:
df=pd.DataFrame(features['MONTH(joined)'].head())
print(df)

           MONTH(joined)
client_id               
25707                 10
26326                  5
26695                  8
26945                 11
29841                  8

In [32]:
# Show a feature with a depth of 1
df2=pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))
print(df2)

           MEAN(loans.loan_amount)
client_id                         
25707                  7963.950000
26326                  7270.062500
26695                  7824.722222
26945                  7125.933333
29841                  9813.000000
32726                  6633.263158
32885                  9920.400000
32961                  7882.235294
35089                  6939.200000
35214                  7173.555556

In [33]:
# Show a feature with a depth of 2
df3=pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))
print(df3)

           LAST(loans.MEAN(payments.payment_amount))
client_id                                           
25707                                     293.500000
26326                                     977.375000
26695                                    1769.166667
26945                                    1598.666667
29841                                    1125.500000
32726                                     799.500000
32885                                    1729.000000
32961                                     282.600000
35089                                     110.400000
35214                                    1410.250000

In [34]:
# Perform deep feature synthesis without specifying primitives;不指定时使用所有特征基元（所有类型的转换和聚合操作）
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [36]:
print(features.iloc[:, 4:].head())

           STD(loans.loan_amount)  STD(loans.rate)  MAX(loans.loan_amount)  \
client_id                                                                    
25707                 4149.486062         2.484186                   13913   
26326                 4393.666631         2.057142                   13464   
26695                 4196.462499         1.561659                   14865   
26945                 4543.621769         1.619717                   14593   
29841                 4209.224171         2.122904                   14837   

           MAX(loans.rate)  SKEW(loans.loan_amount)  SKEW(loans.rate)  \
client_id                                                               
25707                 9.44                -0.186352          0.735470   
26326                 6.73                 0.149658          1.181651   
26695                 6.51                 0.168879          0.896574   
26945                 5.65                 0.174492         -0.002227   
29841          